# 06_workflow

In this **final tutorial**, we explore the automated workflow that ties together everything we learned so far:
- Building designs
- Sweeping parameters
- Running simulations
- Storing results

We'll use the `execute_workflow` function and a `WorkflowConfig` object to show how these pieces fit in a single pipeline.

## Table of Contents
1. [Overview](#overview)
2. [Simple Example](#simple-example)
3. [Full Example](#full-example)
4. [Loading and Saving Configurations](#loading-and-saving-configurations)
5. [Conclusion](#conclusion)

## 1. Overview <a name="overview"></a>

The **Workflow** in `pysubmit` orchestrates the following sequence of actions:

1. A new result directory is created (or an existing one is used).
2. For each set of parameters (like geometry dimensions or frequencies):
   - The parameters are saved for record-keeping, and used to build or modify the design.
   - One or more simulations are run (e.g., HFSS eigenmode analysis).
   - The simulation outputs (e.g., frequencies, Q-factors) are serialized to JSON.
3. Once all parameter sets are processed, the workflow aggregates these results based on a user-specified grouping configuration.
4. The final aggregated data is saved for subsequent analysis or archiving.

We’ll see how `WorkflowConfig` and `execute_workflow` automate these steps.

## 2. Simple Example <a name="simple-example"></a>

In the simplest scenario, we open `simple_design_readout.aedt` and execute just one simulation (`EigenmodeAnalysis`).

In [5]:
from pysubmit import WorkflowConfig, SessionParameters, EigenmodeAnalysis
config = WorkflowConfig(
    session_parameters=SessionParameters(file_path='./designs/simple_design.aedt', design_name='my_design'),
    simulations={'classical': EigenmodeAnalysis(setup_name='Setup1', design_name='my_design')}
)

To run this configuration:

In [6]:
from pysubmit import execute_workflow

execute_workflow(config)

PyAEDT INFO: Parsing C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\designs\simple_design.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.13.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: File C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\designs\simple_design.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_c3cc4a07-45d3-4311-a37b-c88c1ca2f51d.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 62763
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 1960.
PyAEDT INFO: Project 

The simulation output (JSON file) is saved under something like `results/iterations/iteration_0/classical.json`, where **classical** is just the key for `EigenmodeAnalysis` in `simulations`.

## 3. Full Example <a name="full-example"></a>

Here, we add a sweep over `chip_base_length` (or some design parameter) using a `ScriptBuilder`. This merges multiple features: parameter sweeping, script-based building, and the standard `EigenmodeAnalysis` simulation.

In [7]:
from pysubmit import WorkflowConfig, SessionParameters, EigenmodeAnalysis, DataHandler
from pysubmit.workflow.builder import ScriptBuilder
from pysubmit.workflow.sweep import ZipSweep
from pysubmit import execute_workflow

config_full = WorkflowConfig(
    session_parameters=SessionParameters(file_path='./designs/simple_design.aedt'),
    simulations={'classical': EigenmodeAnalysis(setup_name='Setup1', design_name='my_design')},

    # A list of sweeps. We'll do a ZipSweep for chip_base_length
    builder_sweep=[ZipSweep(parameters={'value': ['10mm', '12mm', '14mm', '16mm']})],

    # The builder to apply to each sweep iteration
    builder=ScriptBuilder(path='./builder_scripts/script_builder_function.py'),

    # A DataHandler to store build_parameters + classical results
    data_handler=DataHandler(grouping_config={'my_aggregation': ('build_parameters', 'classical')})
)

execute_workflow(config_full)

PyAEDT INFO: Parsing C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\designs\simple_design.aedt.
PyAEDT INFO: Python version 3.11.3 (tags/v3.11.3:f3909b8, Apr  4 2023, 23:49:59) [MSC v.1934 64 bit (AMD64)]
PyAEDT INFO: PyAEDT version 0.13.0.
PyAEDT INFO: Initializing new Desktop session.
PyAEDT INFO: File C:\Users\rosengrp\Documents\GitHub\pysubmit\tutorial\designs\simple_design.aedt correctly loaded. Elapsed time: 0m 0sec
PyAEDT INFO: Log on console is enabled.
PyAEDT INFO: Log on file C:\Users\rosengrp\AppData\Local\Temp\pyaedt_rosengrp_c3cc4a07-45d3-4311-a37b-c88c1ca2f51d.log is enabled.
PyAEDT INFO: Log on AEDT is enabled.
PyAEDT INFO: Debug logger is disabled. PyAEDT methods will not be logged.
PyAEDT INFO: Launching PyAEDT with gRPC plugin.
PyAEDT INFO: New AEDT session is starting on gRPC port 62825
PyAEDT INFO: AEDT installation Path C:\Program Files\AnsysEM\v242\Win64
PyAEDT INFO: Ansoft.ElectronicsDesktop.2024.2 version started with process ID 20960.
PyAEDT INFO: Project

By default, each iteration’s parameters are stored under the identifier `build_parameters`, and the simulation results are saved under `classical.json`. The aggregator can merge them into a CSV, creating a column for the `chip_base_length` and relevant simulation results.

## 4. Loading and Saving Configurations <a name="loading-and-saving-configurations"></a>

The workflow configuration can be saved or loaded via YAML, making it easy to run in different environments.

In [8]:
config_full.save_to_yaml('./configs/workflow_config_example.yaml')
loaded_config = WorkflowConfig.load_from_yaml('./configs/workflow_config_example.yaml')
print(loaded_config.model_dump_json(indent=4))

{
    "session_parameters": {
        "file_path": "designs\\simple_design.aedt",
        "design_name": "temp",
        "version": "2024.2",
        "non_graphical": false,
        "new_desktop": true,
        "close_on_exit": true
    },
    "simulations": {
        "classical": {
            "type": "eigenmode",
            "setup_name": "Setup1",
            "design_name": "my_design",
            "cores": 4,
            "gpus": 0,
            "setup_parameters": {}
        }
    },
    "data_handler": {
        "root_directory": ".",
        "results_dirname": "results_6",
        "iterations_dirname": "iterations",
        "aggregations_dirname": "aggregations",
        "grouping_config": {
            "my_aggregation": [
                "build_parameters",
                "classical"
            ]
        },
        "counter": 3,
        "last_iteration_path": "results_6\\iterations\\iteration_3",
        "overwrite": false,
        "use_unique": true,
        "results_directory

## 5. Conclusion <a name="conclusion"></a>

In this notebook, we:
- Showed a **simple workflow** (single simulation, no sweep).
- Explored a **full example** combining parameter sweeps (`ZipSweep`), script-based building, multiple simulations, and data aggregation.
- Demonstrated how to **save and load** `WorkflowConfig` from YAML.

This wraps up the end-to-end process: you now have all the pieces to automate design building, parameter sweeping, simulation, and data handling. For advanced usage, explore chaining multiple sweeps or custom aggregator settings to tailor your own workflows.